In [1]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

import pandas, seaborn, numpy, collections, sefara, varcode, uncertainties, numpy, statsmodels, pickle, scipy
%matplotlib inline
from matplotlib import pyplot
import topiary
import mhctools
import matplotlib_venn
import intervaltree
import statsmodels.robust
import mhcflurry
from varcode.common import memoize
pandas.set_option('display.max_columns', None)
import scipy, sys, sklearn
import sklearn.utils
import matplotlib
#import common as c
import sklearn.tree
import pymc3
from scipy import optimize
import glob


def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

matplotlib.rc("savefig", dpi=800)
#matplotlib.rc("savefig", dpi=72)
matplotlib.rc('text', usetex=False)
#reload(c)

%matplotlib inline
%config InlineBackend.figure_format = 'png'

import pystan
import pyfaidx



/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [2]:
# C elegans genome for normalization constants
# THIS IS SLOW
import Bio, gzip
c_elegans_seq = Bio.SeqIO.parse(gzip.open("/Users/tim/sinai/git/ovarian-paired-analyses//data/original/Caenorhabditis_elegans.WBcel235.dna_sm.toplevel.fa.gz"), 'fasta')
c_elegans_seq

counts = collections.Counter()
for record in c_elegans_seq:
    for i in range(len(record.seq) - 3):
        if (i % 100000 == 0):
            print("%d / %d = %f%%" % (i, len(record.seq), i * 100.0 / len(record.seq)))
        sub = str(record.seq[i:i+3])
        if sub == sub.upper():
            counts[sub] += 1
            
def canonicalize(trinucleotide):
    (left, middle, right) = trinucleotide
    if middle in ('A', 'G'):
        return pyfaidx.complement(right) + pyfaidx.complement(middle) + pyfaidx.complement(left)
    return trinucleotide

canonicalized_counts = [(canonicalize(key), count) for (key, count) in counts.items()]
genome_counts = pandas.DataFrame(
    canonicalized_counts, columns=["context", "count"]).groupby("context")["count"].sum().to_frame()
genome_counts

0 / 15072434 = 0.000000%
100000 / 15072434 = 0.663463%
200000 / 15072434 = 1.326926%
300000 / 15072434 = 1.990389%
400000 / 15072434 = 2.653851%
500000 / 15072434 = 3.317314%
600000 / 15072434 = 3.980777%
700000 / 15072434 = 4.644240%
800000 / 15072434 = 5.307703%
900000 / 15072434 = 5.971166%
1000000 / 15072434 = 6.634628%
1100000 / 15072434 = 7.298091%
1200000 / 15072434 = 7.961554%
1300000 / 15072434 = 8.625017%
1400000 / 15072434 = 9.288480%
1500000 / 15072434 = 9.951943%
1600000 / 15072434 = 10.615406%
1700000 / 15072434 = 11.278868%
1800000 / 15072434 = 11.942331%
1900000 / 15072434 = 12.605794%
2000000 / 15072434 = 13.269257%
2100000 / 15072434 = 13.932720%
2200000 / 15072434 = 14.596183%
2300000 / 15072434 = 15.259646%
2400000 / 15072434 = 15.923108%
2500000 / 15072434 = 16.586571%
2600000 / 15072434 = 17.250034%
2700000 / 15072434 = 17.913497%
2800000 / 15072434 = 18.576960%
2900000 / 15072434 = 19.240423%
3000000 / 15072434 = 19.903885%
3100000 / 15072434 = 20.567348%
3200000

,count
context,
ACA,2811348
ACC,1351201
ACG,1287484
ACT,2361788
ATA,3223133
ATC,2553212
ATG,2644784
ATT,5578505
CCA,1986970


In [7]:
rates = (genome_counts / genome_counts.sum())
rates.columns = ["rate"]
rates.to_csv("../data/c_elegans.trinucleotides.csv")

In [ ]:
'''
signatures_df = pandas.read_table("../data/signatures_probabilities.txt")
signatures_df["Somatic Mutation Type"] = signatures_df["Somatic Mutation Type"].str.replace("[", "(").str.replace("]", ")")
signatures_df.index = signatures_df["Somatic Mutation Type"]
for column in signatures_df.columns:
    if 'signature' not in column.lower():
        del signatures_df[column]
signatures_df #.sum(axis=0)
'''

In [ ]:
'''
signatures_df["Cisplatin"] = fractions
signatures_df["Cisplatin"] = signatures_df["Cisplatin"].fillna(0)

def sort_key(context):
    return (context.split(">")[0][-1],context.split(">")[1][0], context  )
signatures_df = signatures_df.ix[sorted(signatures_df.index, key=sort_key)]


signatures_df.to_csv("../data/all_signatures.csv")
signatures_for_deconstructsigs_df = signatures_df.T.copy()
signatures_for_deconstructsigs_df.columns = [
    x.replace("(", "[").replace(")", "]") for x in signatures_for_deconstructsigs_df.columns
]
signatures_for_deconstructsigs_df.to_csv("../data/all_signatures_for_deconstructsigs.csv")

signatures_df #.sum(axis=0)
'''

In [ ]:
'''
seaborn.set_context('poster')
seaborn.set_style("white")
pyplot.figure(figsize=(8,8))

df = signatures_df.reset_index()
grid=(3,2)
positions = []
for row in range(grid[0]):
    for col in range(grid[1]):
        positions.append((row, col))

prev_axes = []
for ((row, col), kind) in zip(positions, sorted(set(df["Somatic Mutation Type"].str.slice(2,5)))):
    sharex = sharey = None
    if col > 1:
        sharey = prev_axes[-1]
    if row != 0:
        sharex = prev_axes[-grid[1]]
        
    ax = pyplot.subplot2grid(grid, (row, col), sharex=sharex, sharey=sharey)
    prev_axes.append(ax)
    
    sub_df = df[df["Somatic Mutation Type"].str.contains(kind)]
    
    seaborn.barplot(ax=ax,
                    data=sub_df,
                    y="Cisplatin",
                    x="Somatic Mutation Type")
    pyplot.xticks(numpy.arange(sub_df.shape[0]), sub_df["Somatic Mutation Type"].str.replace("(%s)" % kind, "$\cdot$"))
    pyplot.xticks(rotation=90)
    pyplot.ylabel("")
    pyplot.xlabel("")
    pyplot.title(kind.replace(">", "$\\rightarrow$"))
    pyplot.ylim(ymin=0, ymax=0.5)
    #ax.xaxis.grid('on')
   
    #ax.label_outer()
    for label in ax.get_yticklabels():
        label.set_visible(ax.is_first_col())
        
#pyplot.suptitle("Cisplatin signature")
pyplot.tight_layout()
'''